In [1]:
# Use nest_asyncio to allow nested event loops in notebooks
!pip install nest_asyncio

In [2]:
import os
import sys

import nest_asyncio

# Get the current working directory of the notebook
# This will be '.../maxtext/MaxText/scratch_code'
current_dir = os.getcwd()

# Navigate two levels up to get to the project root 'maxtext'
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))

# Add the project root to the system path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added '{project_root}' to sys.path")

nest_asyncio.apply()

Added '/home/mazumdera_google_com/maxtext' to sys.path


Sanity test to ensure we have TPU devices accessible

In [3]:
import jax
jax.distributed.initialize() # distributed.initialize should only be called once.
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=4, process_index=0, coords=(0,2,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(1,2,0), core_on_chip=0),
 TpuDevice(id=6, process_index=0, coords=(0,3,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,3,0), core_on_chip=0)]

Import MaxText and its components

In [4]:
import MaxText as mt
from MaxText import pyconfig
from MaxText import maxtext_utils
import numpy as np
from MaxText.input_pipeline import _input_pipeline_utils
import os
import datetime
from MaxText.globals import PKG_DIR
from MaxText import max_logging
from MaxText import common_types
import jax
from MaxText import train_utils
from MaxText.train import train_step, eval_step
from MaxText.data_loader import DataLoader
from MaxText import checkpointing
from flax import linen as nn
import tqdm


2025-08-26 23:56:47.230922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756252607.244333 1781995 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756252607.248280 1781995 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756252607.260243 1781995 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756252607.260256 1781995 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756252607.260258 1781995 computation_placer.cc:177] computation placer alr

In [ ]:
# Replace path to your Llama3.1-8b checkpoint for the `load_parameters_path` argument.
config = pyconfig.initialize(
    ["", "../configs/base.yml"], 
    per_device_batch_size=1.0,
    run_name="test",
    max_target_length=4,
    max_prefill_predict_length=4,
    tokenizer_type="tiktoken",
    tokenizer_path="assets/tokenizer_llama3.tiktoken/",
    load_parameters_path="gs://maxtext-model-checkpoints/llama3.1-8b/2025-01-23-19-04/scanned/0/items",
    model_name="llama3.1-8b",
    async_checkpointing=False,
    dataset_type="hf",
    hf_access_token="${HF_ACCESS_TOKEN}",

)

model = mt.from_config(config)
mesh = model.mesh
# init_rng = jax.random.PRNGKey(config.init_weights_seed)
# state, _ = maxtext_utils.setup_decode_state(model, config, init_rng, mesh, None)


(
      init_rng,
      checkpoint_manager,
      state_mesh_shardings,
      model,
      mesh,
      learning_rate_schedule,
      data_iterator,
      eval_data_iterator,
      state,
  ) = train_utils.setup_train_loop(config, None)




Updating keys from env and command line: ['run_name', 'model_name', 'load_parameters_path', 'async_checkpointing', 'tokenizer_path', 'tokenizer_type', 'per_device_batch_size', 'dataset_type', 'hf_access_token', 'max_target_length', 'max_prefill_predict_length']
Running Model: llama3.1-8b
Updating following parameters in config

base_emb_dim: 4096
base_num_query_heads: 32
base_num_kv_heads: 8
base_num_decoder_layers: 32
base_mlp_dim: 14336
head_dim: 128
mlp_activations: ['silu', 'linear']
vocab_size: 128256
enable_dropout: False
logits_via_embedding: False
normalization_layer_epsilon: 1e-05
rope_max_timescale: 500000
decoder_block: llama2
Updating keys from model: ['base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_num_decoder_layers', 'base_mlp_dim', 'head_dim', 'mlp_activations', 'vocab_size', 'enable_dropout', 'logits_via_embedding', 'normalization_layer_epsilon', 'rope_max_timescale', 'decoder_block']
Jax distributed system is already initialized.
Not using emergency

AssertionError: hf_path can't be empty when dataset_type=hf

In [ ]:
p_train_step, p_eval_step = train_utils.jit_train_and_eval_step(
      config, model, mesh, state, state_mesh_shardings, train_step, eval_step, eval_data_iterator
  )

In [ ]:
assert eval_data_iterator

def evaluate(eval_data_iterator, state, rng):
    eval_step_count = 0
    # pylint: disable=not-callable
    for eval_batch in eval_data_iterator:
        if config.eval_steps > 0 and eval_step_count >= config.eval_steps:
            break
        with mesh, nn_partitioning.axis_rules(config.logical_axis_rules):
            eval_metrics = p_eval_step(state, eval_batch, rng)
            print(f"evaluation/total_loss = {eval_metrics["scalar"].get("evaluation/total_loss", 0.0)} at eval step = {eval_step_count}")

In [ ]:
start_step = int(state.step)  # this is the start_step for training
data_loader = DataLoader(config, mesh, data_iterator, None)

In [ ]:


last_step_completion = datetime.datetime.now()
nextrng = jax.jit(jax.random.fold_in)(init_rng, start_step)

evaluate(eval_data_iterator, state, nextrng)

for step in tqdm(np.arange(start_step, config.steps)):

    example_batch = data_loader.load_next_batch()
    with mesh, nn_partitioning.axis_rules(config.logical_axis_rules):
        state, metrics = p_train_step(state, example_batch, nextrng)

    step_time_delta = datetime.datetime.now() - last_step_completion
    last_step_completion = datetime.datetime.now()
    nextrng = jax.jit(jax.random.fold_in)(init_rng, step)

checkpointing.maybe_save_checkpoint(checkpoint_manager, state, config, data_iterator, step)

print("Training complete.")

evaluate(eval_data_iterator, state, nextrng)  


In [ ]:
input_ids = source_tokenizer.encode(config.prompt)
ids = np.asarray(input_ids, dtype=np.int32)
s = (config.global_batch_size_to_train_on, config.max_target_length)
decoder_segment_ids = np.zeros(s) + common_types.DECODING_ACTIVE_SEQUENCE_INDICATOR
decoder_positions = np.stack(
    [np.arange(config.max_target_length, dtype=np.int32) for _ in range(config.global_batch_size_to_train_on)]
)


ids = np.stack([ids for _ in range(config.global_batch_size_to_train_on)])
max_logging.log(f"input_ids={input_ids}, \nids={ids}, \ndecoder_segment_ids = {decoder_segment_ids}, \ndecoder_positions= {decoder_positions}")


Run a forward pass

In [ ]:
full_train_logits = model.apply(
          state.params,
          ids,
          decoder_positions,
          decoder_segment_ids,
          enable_dropout=False,
          rngs={"aqt": init_rng},
      )
full_train_logits = jax.experimental.multihost_utils.process_allgather(full_train_logits)
max_logging.log(f"{full_train_logits[0, 0, :]=}")

Check the logits

In [ ]:
selected_logits = jax.lax.dynamic_slice(
        full_train_logits,
        (0, 0, full_train_logits.shape[2]-1, 0),
        (1, 1, 1, full_train_logits.shape[3])
    )

In [ ]:
# Consider the greedily sampled token
init_rng, new_rng = jax.random.split(init_rng)
first_generated_token = inference_utils.sampling(
        selected_logits,
        new_rng,
        config.decode_sampling_strategy, #"greedy"
    )

In [ ]:
first_generated_token.item()

In [ ]:
source_tokenizer.decode([first_generated_token.item()])